## OOP: Statische Methoden und Klassenmethoden

In [ ]:
help(staticmethod)

#### Klasse für Schachfiguren mit statischer Methode zur Umrechnung von Koordinaten

z.B. b3 => (1,2)

In [ ]:
class Figure:
    @staticmethod
    def to_cartesian(chess_coordinates):
        """
        Konvertiert Schachbrettkoordinaten wie "b3" in kartesische Koordinaten: "b3" => (1,2)
        :param chess_coordinates: str of length 2
        :return: tuple of length 2
        """
        column = "abcdefgh".find(chess_coordinates[0])
        row = "12345678".find(chess_coordinates[1])
        return column, row


knight = Figure()

# 2 Möglichkeiten für den Aufruf der statischen Methode
# 1. Über Klasse Figure
print(Figure.to_cartesian("b7"))

# 2. Über Instanz knight
print(knight.to_cartesian("a3"))

Beachten Sie, dass statische Methoden keinen formalen Parameter `self` haben. Mit `self` greift man schließlich auf die Instanz(-attribute/methoden) zu.
Die statische Methode `to_cartesian(chess_coordinates)` hätte auch außerhalb einer Klasse als Funktion definiert werden können, es ist aber sicherlich sinnvoll, eine solche Methode in der Klasse Figure zusammen mit weiterer Funktionalität zu bündeln.

Wir ergänzen die Implementierung um eine Methode, die alle möglichen Schachkoordinaten ausgibt:

In [ ]:
class Figure:
    @staticmethod
    def to_cartesian(chess_coordinates):
        """
        Konvertiert Schachbrettkoordinaten wie "b3" in kartesische Koordinaten: "b3" => (1,2)
        :param chess_coordinates: str of length 2
        :return: tuple of length 2
        """
        column = "abcdefgh".find(chess_coordinates[0])
        row = "12345678".find(chess_coordinates[1])
        return column, row

    @staticmethod
    def get_chess_coordinates():
        """
        Berechnet alle möglichen Schachkoordinaten
        :return: Liste aller Schachkoordinaten "a1", ..., "h8"
        """
        return [col + row for col in "abcdefgh" for row in "12345678"]


print(Figure.get_chess_coordinates())

Ein Nachteil dieser Implementierung ist, dass wir die x-Koordinaten "abcdefgh" sowie y-Koordinaten "12345678" in unterschiedlichen Methoden explizit hinschreiben müssen.
Eigentlich handelt es sich aber hierbei um Konstanten der Klasse!

Wir lösen das nun schöner mit Klassenmethoden.

In [ ]:
help(classmethod)

### Klassenmethoden

In [ ]:
class Figure:
    CHESS_COLUMNS = "abcdefgh"
    CHESS_ROWS = "12345678"

    @classmethod
    def to_cartesian(cls, chess_coordinates):
        """
        Konvertiert Schachbrettkoordinaten wie "b3" in kartesische Koordinaten: "b3" => (1,2)
        :param chess_coordinates: str of length 2
        :return: tuple of length 2
        """
        column = cls.CHESS_COLUMNS.find(chess_coordinates[0])
        row = cls.CHESS_ROWS.find(chess_coordinates[1])
        return column, row

    @classmethod
    def get_chess_coordinates(cls):
        """
        Berechnet alle möglichen Schachkoordinaten
        :return: Liste aller Schachkoordinaten "a1", ..., "h8"
        """
        return [col + row for col in cls.CHESS_COLUMNS for row in cls.CHESS_ROWS]

In [ ]:
knight = Figure()
# Aufruf über Klasse ...
print(Figure.get_chess_coordinates())
#... oder Instanz möglich
print(knight.get_chess_coordinates())

Wir erstellen nun separate Klassen für unterschiedliche Schachfiguren.
Zunächst fügen wir (vgl. Folien) einen Zähler für die erzeugten Figuren ein.
Danach erweitern wir die Klasse `Figure` um eine Methode `get_name()` oder `move()`. Diese müssen für unterschiedliche Figuren unterschiedlich implementiert werden.
Wir lösen dies hier mittels Vererbung und Polymorphismus.
(Die Koordinaten-Methoden lassen wir hier für eine bessere Übersicht weg.)

In [ ]:
class Figure:
    """
    Oberklasse für alle Schachfiguren
    """
    _count = 0

    def __init__(self):
        Figure._count += 1

    @classmethod
    def get_count(cls):
        return Figure._count


class Knight(Figure):
    def __init__(self):
        super().__init__()


knight_1 = Knight()
knight_2 = Knight()

# Aufrufmöglichkeiten der Methode get_count()
print(Knight.get_count())
print(knight_1.get_count())
print(Figure.get_count())

In [7]:
class Figure:
    """
    Oberklasse für alle Schachfiguren
    """
    _count = 0  # Anzahl der Figuren auf dem Schachbrett
    _name = ""  # Name muss in Unterklassen vergeben werden

    def __init__(self):
        Figure._count += 1

    @classmethod
    def get_count(cls):
        return Figure._count

    @classmethod
    def get_name(cls):
        return cls._name


class Knight(Figure):
    """
    Klasse für Springer
    """
    _name = "Springer"

    def __init__(self):
        super().__init__()

In [8]:
knight_b1 = Knight()
# Zugriff über Instanz
print(f"Instanz knight_b1: Attribut _name = {knight_b1.get_name()}")
# Zugriff über Klasse
print(f"Klasse Knight: Attribut _name = {Knight.get_name()}")
print(f"Klasse Figure: Attribut _name ={Figure.get_name()}")

Instanz knight_b1: Attribut _name = Springer
Klasse Knight: Attribut _name = Springer
Klasse Figure: Attribut _name =


Die Klassenmethode `get_name()` reagiert dynamisch auf die vererbten und ggf. überschriebenen Attribute:
Es wird beim Aufruf der Klassenmethode `Knight.get_name()` der Wert des Attributes `name` der Klasse `Knight` zurückgegeben und nicht der Wert von `name` der Klasse `Figure`, in der die Klassenmethode definiert wurde.

Dies ist ein wichtiger Unterschied zum Verhalten statischer Methoden in Java: Der folgende vergleichbare Java-Code gibt "Figure" aus!

```java
class Figure {
    static String name = "Figure";

    static String getName() {
        return name;
    }
}

class Knight extends Figure {
    static String name = "Springer";
}

public class Main {
    public static void main(String[] args) {
        System.out.println(Knight.getName()); // Ausgabe "Figure" !
    }
}
```

Klassenmethoden können auch dazu genutzt werden, eine alternative Schnittstelle zur Erzeugung einer (speziellen) Klasseninstanz anzubieten.
Im folgenden Beispiel sollen alle Schachfiguren Instanzen der Klasse `Figure` sein. Um unterschiedliche Figuren unterscheiden zu können, müssen wir uns dann in Instanz-Attributen die Figur-Art (Bauer, Dame,...) und die Farbe (schwarz oder weiß) merken.

In [11]:
class Figure:
    _count = 0

    def __init__(self, figure_type, color):
        self.type = figure_type
        self.color = color
        Figure._count += 1

    @classmethod
    def create_black_knight(cls):
        return cls("knight", "black")

    @classmethod
    def get_count(cls):
        return Figure._count


# 1. schwarzer Springer (Instanz-Erzeugung wie bisher):
knight_black_1 = Figure("knight", "black")
# 2. schwarzer Springer (alternative Instanz-Erzeugung):
knight_black_2 = Figure.create_black_knight()

print(Figure.get_count())

2
2
